In [1]:
using Statistics
using Plots

In [2]:
ppal_dir = pwd()

function init_lattice(L, N, l)

    FCWS = []

    grid = zeros((L, L))
    
    idx = 0

    while idx < N

        x_head = rand(1 : (L - 1))

        x_tail = x_head + l - 1

        row = rand(1 : L)

        positions = [(row, mod(j, 1 : L)) for j in x_head : x_tail]

        validator = true

        #Check if any of these positions have been taken
        for pos in positions

            if grid[positions[1][1], positions[1][2]] == 1

                validator = false
                
            end

        end

        #if any position has been taken don't do anything
        if validator == false
            #DO nothing

        #If all the positions are free then take it
        else
            
            idx += 1

            for j in x_head : x_tail

                grid[row, mod(j, 1 : L)] = 1
                
            end

            append!(FCWS, [positions])

        end
    end

    return FCWS, grid

end


function move_walkers(L, N, l, FCWS, grid)

    N_mov = 0

    k = 0
    
    @fastmath @inbounds for fcw in FCWS

        k += 1
        
        new_pos = [(0, 0) for j in 1 : l]

        #Choose 1 free nearest neighbours poisition
        i = fcw[1][1]
        j = fcw[1][2]

        lattice_pos = [(i, mod(j+1, 1:L)), (i, mod(j-1, 1:L)), (mod(i+1, 1:L), j), (mod(i-1, 1:L), j)]
        
        free_pos = []

        for pos in lattice_pos
            
            if grid[convert(Int32, pos[1]), convert(Int32, pos[2])] == 0
                
                append!(free_pos, [pos])
                
            end
        end
        

        #Only move if not occupied
        if length(free_pos) != 0
            
            #Move the head to the chosen position
            move_to = free_pos[rand(1:length(free_pos))]

            N_mov += 1
            
            #Update lattice for the tail
            grid[fcw[end][1], fcw[end][2]] = 0

            #Move the other particles following the head
            for i in 2 : l

                new_pos[i] = fcw[i-1]
                    
            end
            
            #Move the head
            new_pos[1] = move_to
            
            #Update FCW position
            FCWS[k] = new_pos
       
            #Update lattice for head
            grid[move_to[1], move_to[2]] = 1
                    
        end
    end
    
    mob = N_mov / N
    
    return mob, grid, FCWS
        
end

function simulate(L, N, l, t)

    FCWS, grid = init_lattice(L, N, l)
    
    heatmap(grid)

    mobility_arr = zeros(t)

    for k in 1 : t

        percentage = round(k/t * 100, digits=2)
        
        #println(percentage, " %", " done")

        mobility, grid, FCWS = move_walkers(L, N, l, FCWS, grid)
        
        @inbounds mobility_arr[k] = mobility
            
    end

    return mobility_arr, grid
        
end

simulate (generic function with 1 method)

# Single density simulation

In [3]:
function single_density(L, N, l, t, folder="Data")

    t0 = time_ns() #Start simulation

    mobility, grid = simulate(L, N, l, t)

    tf = time_ns() #Simulation finished

    ET = round((tf - t0) * 1e-9, sigdigits=3) #Compute elapsed time
    
    #Create folder if doesn't exist
    if ! isdir(folder)
        mkdir(folder)
    end
    
    #Choose folder
    cd(folder)
    
    #Write parameters used to file
    f_parameters = open("parameters_used.txt", "w")
    
    println(f_parameters, "L: ", L)
    println(f_parameters, "l: ", l)
    println(f_parameters, "N: ", N)
    println(f_parameters, "t: ", t)
    
    println(f_parameters, "\nElapsed time: ", ET)
    
    close(f_parameters)
    
    cd(ppal_dir) #Return to home directory
    
    println("Elapsed time: ", ET, " s")
    
    return mobility, grid

end

single_density (generic function with 2 methods)

# Several densities simulations

In [4]:
function several_densities(L, l, t, eq_t, times, densities, folder="Data")

    t0 = time_ns()

    k = 0
    
    if ! isdir(folder)
        mkdir(folder)
    end
    
    cd(folder)
    
    f = open("mobility.txt", "w")
    
    println(f, "#<M(t)>\tρ")
    println("#mobility\tdensity")
    
    #Simulate and compute mobility for each density
    for density in densities

        k += 1

        N = convert(Int, round(L^2 * density / l, digits=0))
        
        mean_mob = 0.
        
        #Average over "times" realisations
        for i in 1 : times
    
            mobility, grid = simulate(L, N, l, t)

            mean_mob += mean(mobility[eq_t : end])
            
        end
        
        mean_mob = mean_mob / times
        
        println(f, mean_mob, "\t", density)
        println(mean_mob, "\t", density)

    end
    
    close(f)
    
    tf = time_ns()
    
    ET = round((tf - t0) * 1e-9, sigdigits=3)
    
    f_parameters = open("parameters_used.txt", "w")
    
    println(f_parameters, "L: ", L)
    println(f_parameters, "l: ", l)
    println(f_parameters, "t: ", t)
    println(f_parameters, "Eq_t: ", eq_t)
    println(f_parameters, "times: ", times)
    
    println(f_parameters, "\nElapsed time: ", ET)
    
    close(f_parameters)
    
    cd(ppal_dir)

    println("Elapsed time: ", ET, " s")
    
end

several_densities (generic function with 2 methods)

# Run simulation 

In [5]:
L = 100
l = 1

t = 10^4
eq_t = 10^3

times = 5

densities = collect(0.01: 0.01 : 0.8)

foldername = "l_$l"

several_densities(L, l, t, eq_t, times, densities, foldername)

#mobility	density
1.0	0.01
1.0	0.02
0.9999998518683109	0.03
0.9999995556049328	0.04
0.9999981779802244	0.05
0.9999953338517942	0.06
0.9999919056612759	0.07
0.9999871958671258	0.08
0.9999774839832613	0.09
0.9999650261082103	0.1
0.9999504095504541	0.11
0.9999273414065104	0.12
0.9998996692675174	0.13
0.9998580316472772	0.14
0.9998130726215605	0.15
0.9997495972669705	0.16
0.9996795519452089	0.17
0.9995843795133876	0.18
0.9994779878259141	0.19
0.9993523164092876	0.2
0.9991965337184749	0.21
0.9990256234155807	0.22
0.9988076107485643	0.23
0.9985653538495729	0.24
0.9982880213309633	0.25
0.997952415543572	0.26
0.9975821534232823	0.27
0.9971495071976131	0.28
0.9966857398986319	0.29
0.9961354368033181	0.3
0.9955228272127471	0.31
0.9948353238529049	0.32
0.9940817484925916	0.33
0.9931796597763647	0.34
0.9922597298712839	0.35
0.9912104519250947	0.36
0.9900310596119951	0.37
0.9887201714429391	0.38
0.9873011374804509	0.39
0.9857325297189202	0.4
0.9840083405366883	0.41
0.9821139873347405	0.42
0.9800611